In [1]:
%matplotlib qt

import numpy as np
import scipy.linalg as linalg
from matplotlib.pyplot import *
import pickle
from quadcopter.util import *

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 160
rcParams['pgf.rcfonts'] = False
rcParams['lines.linewidth'] = 1.2


colors = {
    'reference': 'black',
    'ekf': 'red',
    'mhe1': ['green', 'olive', 'jungle green'],
    'mhe2': ['blue', 'amethyst', 'water blue']
}


def save_plot(plot_name):
    savefig(f"C:\\Users\\Robert\\Seafile\\Uni\\Studienarbeit\\Arbeit\\plots\\{plot_name}.pgf")


def to_deg(rad):
    return rad / np.pi * 180.0


def load_results(file_name):
    complete_path = f"C:\\dev\\mhe_pymoskito\\quadcopter\\results\\simulation\\{file_name}.pmr"
    with open(complete_path, "rb") as f:
        return pickle.load(f)['results']
    

class ExperimentData:
    def __init__(self, ex_id, raw):
        self.ex_id = ex_id
        self.raw = raw
    
    @property
    def time(self):
        return self.raw['time']
    
    @property
    def ref_pos(self):
        return self.raw['Observer'][:, :3].T
               
    @property
    def ref_ori(self):
        return self.raw['Observer'][:, 3:6].T
    
    @property
    def obs_ori(self):
        return self.raw['Observer'][:, 6:9].T
               
    @property
    def obs_error(self):
        return self.raw['Observer'][:, 9:12].T
    
    @property
    def obs_gyro_bias(self):
        return self.raw['Observer'][:, 12:15].T
    
    @property
    def obs_acc_bias(self):
        return self.raw['Observer'][:, 15:18].T
    
    @property
    def meas_gyro(self):
        return self.raw['Observer'][:, 18:21].T
    
    @property
    def meas_acc(self):
        return self.raw['Observer'][:, 21:24].T
    
    
def load_experiment(ex_id, resample_steps=None, start_time=None, end_time=None) -> ExperimentData:
    raw = load_results(ex_id)
    
    start_index = None
    end_index = None
    
    if start_time is not None:
        start_index = time_to_index(start_time, raw['time'])
        
    if end_time is not None:
        end_index = time_to_index(end_time, raw['time'])
    
    raw['time'] = raw['time'][start_index:end_index:resample_steps]
    raw['Solver'] = raw['Solver'][start_index:end_index:resample_steps]
    raw['Observer'] = raw['Observer'][start_index:end_index:resample_steps]
    raw['Model'] = raw['Model'][start_index:end_index:resample_steps]
    return ExperimentData(ex_id, raw)

In [14]:
uav3_ekf_no_error = load_experiment('uav3_ekf_no_error')
uav3_ekf_with_error = load_experiment('uav3_ekf_with_error')
uav3_mhe_no_error = load_experiment('uav3_mhe12_no_error')
uav3_mhe_with_error = load_experiment('uav3_mhe12_with_error')

room3_ekf_no_error = load_experiment('room3_ekf_no_error')
room3_mhe_no_error = load_experiment('room3_mhe12_no_error')

ref = room3_ekf_no_error
experiments = [room3_ekf_no_error, room3_mhe_no_error]

figure('Roll and Pitch')
subplot(221)
plot(ref.time, ref.ref_ori[0], color=colors['reference'])
for ex in experiments:
    plot(ref.time, ex.obs_ori[0])
ylabel("Roll (deg)")
grid()

subplot(222)
for ex in experiments:
    plot(ref.time, ex.obs_error[0])
ylabel("Roll error (deg)")
grid()

subplot(223)
plot(ref.time, ref.ref_ori[1], color=colors['reference'])
for ex in experiments:
    plot(ref.time, ex.obs_ori[1])
ylabel("Pitch (deg)")
grid()

subplot(224)
for ex in experiments:
    plot(ref.time, ex.obs_error[1])
ylabel("Pitch error (deg)")
grid()

tight_layout()

figure('Measurements')
subplot(231)
plot(ref.time, ref.meas_gyro[0])
ylabel("gyro x")
grid()
subplot(232)
plot(ref.time, ref.meas_gyro[1])
ylabel("gyro y")
grid()
subplot(233)
plot(ref.time, ref.meas_gyro[2])
ylabel("gyro z")
grid()

subplot(234)
plot(ref.time, ref.meas_acc[0])
ylabel("acc x")
grid()
subplot(235)
plot(ref.time, ref.meas_acc[1])
ylabel("acc y")
grid()
subplot(236)
plot(ref.time, ref.meas_acc[2])
ylabel("acc z")
grid()

figure('Orientation error norm')
for ex in experiments:
    error_norm = linalg.norm(ex.obs_error, axis=0)
    plot(ref.time, error_norm)

tight_layout()
show()

C:\dev\Python\studienarbeit\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['lmodern'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
